In [334]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
import seaborn as sns
import random

In [335]:
def patch_diameter(patch_degree, eye_monitor_distance):
    # monitor is 509.18mm * 286.42mm
    # patch_degree in degree; eye_monitor_distance in mm
    return 2*np.tan((patch_degree/2)/180*np.math.pi) * eye_monitor_distance # 6.286mm
# Calculate each patch size = patch distance = patch d = 13mm

def distance_between_dots(dot1, dot2, patch_distance):
    '''
    Input two positions dot1, dot2 on the grid, 
    with the knowledge of distance between grid dots,
    calculate distance between two dots.
    '''
    return np.sqrt(((dot1[0]-dot2[0]) * patch_distance)**2 \
                + ((dot1[1]-dot2[1]) * patch_distance)**2)

def generate_one_random_matrix(n_patch_x,n_patch_y,positive_rate):
    n_one = int(np.floor(n_patch_x*n_patch_y*positive_rate))
    nums = np.ones(n_patch_x*n_patch_y)
    nums[:int(n_patch_x*n_patch_y-n_one)] = 0
    np.random.shuffle(nums)    
    array = nums.reshape(n_patch_x,n_patch_y)
    return array, n_one

def get_one_matrix_position(n_patch_x, n_patch_y, positive_rate):
    '''
    Generate a matrix with one and zero with a rate
    '''
    array, n_one = generate_one_random_matrix(n_patch_x,n_patch_y,positive_rate)
    one_ind = np.where(array == 1)
    position = []
    for i in range(n_one):
        position.append([one_ind[0][i], one_ind[1][i]])
    np.random.shuffle(position)
    return array, n_one, position

def generate_one_frame(n_patch_x, n_patch_y, positive_rate, patch_distance, zone_distance): 
    array, n_one, position = get_one_matrix_position(n_patch_x, n_patch_y, positive_rate)
    distance = np.zeros((n_one,n_one))
    for i in range(len(position)):
        for j in range(i+1,len(position)):
            distance[i,j] = distance_between_dots(position[i], position[j], patch_distance) 
            if distance[i,j] < zone_distance:
                array[position[j][0],position[j][1]] = 0
    # plot dots
#     fig= plt.figure(figsize=(5,2))
#     sns.heatmap(array)
    return array

def get_position_after_distance_check(array):
    n_one = np.sum(array)
    one_ind = np.where(array == 1)
    position_after = []
    for i in range(int(n_one)):
        position_after.append([one_ind[0][i], one_ind[1][i]])
    return position_after
    
def turn_on_to_off(array, position_after):
    array_after_off = array.copy()
    minus_choices = random.sample(position_after, int(round(np.sum(array)/2)))
    for minus in minus_choices:
        array_after_off[minus[0],minus[1]] = -1
    return array_after_off

def get_one_frame_dots_position_and_sign(n_patch_y,array_after_off):
    # get dot position and color starting 1
    one_frame_dots = []
    plus_dot_pos = np.where(array_after_off == 1)
    minus_dot_pos = np.where(array_after_off == -1)
    for i in range(len(plus_dot_pos[0])):
        one_frame_dots.append([plus_dot_pos[0][i]*n_patch_y + plus_dot_pos[1][i] + 1, 1]) #get position in one number
    for i in range(len(minus_dot_pos[0])):
        one_frame_dots.append([minus_dot_pos[0][i]*n_patch_y + minus_dot_pos[1][i] + 1, 3]) #get position in one number
    return one_frame_dots

def generate_multi_frames(patch_degree, zone_degree, eye_monitor_distance, n_patch_x, n_patch_y, positive_rate, n_iterations):
    multi_frames = []
    patch_distance = patch_diameter(patch_degree, eye_monitor_distance)
    zone_distance = patch_diameter(zone_degree, eye_monitor_distance)
    for iter in range(n_iterations):
    # generate one frame and turn half of 1 to -1
        array = generate_one_frame(n_patch_x, n_patch_y, positive_rate, patch_distance, zone_distance)
        position_after = get_position_after_distance_check(array)
        array_after_off = turn_on_to_off(array, position_after)
        one_frame_dots = get_one_frame_dots_position_and_sign(n_patch_y,array_after_off)
        multi_frames.append(one_frame_dots)
    return multi_frames

In [336]:
np.random.seed(123)
# generate trials of locally sparse noise with 
multi_frames = generate_multi_frames(patch_degree = 4.6525, 
                                     zone_degree = 23,
                                     eye_monitor_distance = 160,
                                     n_patch_x=20, n_patch_y=35, 
                                     positive_rate = 0.02, 
                                     n_iterations=20000)

In [337]:
# save the list
with open('LocalSparseNoise20000.txt', 'w') as f:
    for item in multi_frames:
        f.write("%s\n" % item)

In [338]:
# validate the noise 
multi_frames_flatten = []
for items in multi_frames:
    for item in items:
        multi_frames_flatten.append(item)
        
# function to get unique values
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list
    
multi_frames_unique = unique(multi_frames_flatten)
multi_frames_unique.sort()
dic_sum = {};
i = 0
for dot_type in multi_frames_unique:
    dic_sum[str(dot_type)] = multi_frames_flatten.count(dot_type)
    i = i+1

In [339]:
sum(dic_sum.values())/len(dic_sum)

113.565

In [340]:
dic_sum

{'[1, 1]': 176,
 '[1, 3]': 160,
 '[2, 1]': 156,
 '[2, 3]': 126,
 '[3, 1]': 168,
 '[3, 3]': 133,
 '[4, 1]': 122,
 '[4, 3]': 140,
 '[5, 1]': 134,
 '[5, 3]': 127,
 '[6, 1]': 141,
 '[6, 3]': 143,
 '[7, 1]': 130,
 '[7, 3]': 127,
 '[8, 1]': 137,
 '[8, 3]': 109,
 '[9, 1]': 125,
 '[9, 3]': 114,
 '[10, 1]': 135,
 '[10, 3]': 132,
 '[11, 1]': 127,
 '[11, 3]': 134,
 '[12, 1]': 125,
 '[12, 3]': 127,
 '[13, 1]': 129,
 '[13, 3]': 152,
 '[14, 1]': 130,
 '[14, 3]': 119,
 '[15, 1]': 114,
 '[15, 3]': 144,
 '[16, 1]': 131,
 '[16, 3]': 138,
 '[17, 1]': 136,
 '[17, 3]': 120,
 '[18, 1]': 127,
 '[18, 3]': 118,
 '[19, 1]': 148,
 '[19, 3]': 124,
 '[20, 1]': 137,
 '[20, 3]': 145,
 '[21, 1]': 126,
 '[21, 3]': 123,
 '[22, 1]': 133,
 '[22, 3]': 158,
 '[23, 1]': 126,
 '[23, 3]': 119,
 '[24, 1]': 144,
 '[24, 3]': 137,
 '[25, 1]': 144,
 '[25, 3]': 119,
 '[26, 1]': 135,
 '[26, 3]': 138,
 '[27, 1]': 137,
 '[27, 3]': 129,
 '[28, 1]': 137,
 '[28, 3]': 150,
 '[29, 1]': 145,
 '[29, 3]': 116,
 '[30, 1]': 139,
 '[30, 3]': 134